In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 64.5 MB/s eta 0:00:00


In [3]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import random
from collections import defaultdict, Counter
from typing import List, Dict, Union

import numpy as np
import implicit
from scipy.sparse import lil_matrix
from tqdm import tqdm
import pandas as pd
import polars as pl

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [5]:
TOP_N = 100
LOCALES = ["IT", "FR", "ES"]
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"
VER = "38"

# parameters for IMF
ALPHA = 12500
LAMBDA = 0.01
FACTORS = 200
N_EPOCHS = 15
SEED = 42
RECENT_N = 2

In [6]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [7]:
def train_and_generate_candidates(df:pd.DataFrame):
    # generate sparse matrix
    unique_user_ids = sorted(df["session_id"].unique())
    unique_item_ids = sorted(df["prev_items"].unique())
    user_id2index = dict(zip(unique_user_ids, range(len(unique_user_ids))))
    item_id2index = dict(zip(unique_item_ids, range(len(unique_item_ids))))

    # overall matrix (for training)
    matrix = lil_matrix((len(unique_user_ids), len(unique_item_ids)))
    for _, row in tqdm(df.iterrows()):
        user_index = user_id2index[row["session_id"]]
        item_index = item_id2index[row["prev_items"]]
        matrix[user_index, item_index] = 1.0
    matrix = matrix.tocsr()

    # recent interaction matrix (for inference)
    matrix_recent = lil_matrix((len(unique_user_ids), len(unique_item_ids)))
    df_recent = df.groupby("session_id").tail(RECENT_N)
    for _, row in tqdm(df_recent.iterrows()):
        user_index = user_id2index[row["session_id"]]
        item_index = item_id2index[row["prev_items"]]
        matrix_recent[user_index, item_index] = 1.0
    matrix_recent = matrix_recent.tocsr()

    # model initialization
    model = implicit.als.AlternatingLeastSquares(
        factors = FACTORS,
        alpha = ALPHA,
        iterations = N_EPOCHS,
        regularization = LAMBDA,
        calculate_training_loss = True,
        random_state = SEED,
    )

    # train model
    model.fit(matrix)

    # generate candidate
    users = []
    candidates = []
    imf_scores = []
    for user_id, user_index in tqdm(user_id2index.items()):
        item_indexes, scores = model.recommend(user_index, matrix_recent[user_index], N=TOP_N, filter_already_liked_items=True)
        for item_index, score in zip(item_indexes, scores):
            users.append(user_id)
            candidates.append(unique_item_ids[item_index])
            imf_scores.append(score)

    candidates = pd.DataFrame({
        "session_id": users,
        "candidate_item": candidates,
        "imf_score_to_recent_intereaction_n2": imf_scores,
    })

    candidates = pl.from_pandas(candidates)
    candidates = candidates \
    .sort(["session_id", "imf_score_to_recent_intereaction_n2"], descending=[False, True]) \
    .with_columns(pl.col("imf_score_to_recent_intereaction_n2").rank(descending=True, method="min").over("session_id").alias("imf_rank_to_recent_intereaction_n2"))
    candidates = candidates.to_pandas()

    return candidates

## For local train/eval

In [8]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2_augmented.parquet")
test = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_leftover.parquet")

In [9]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()

In [10]:
%%time
candidates = train_and_generate_candidates(session_df)

1595272it [01:28, 17962.13it/s]
911508it [00:50, 17900.75it/s]


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 455754/455754 [30:50<00:00, 246.28it/s]


CPU times: user 2h 42min 33s, sys: 1h 40min 39s, total: 4h 23min 13s
Wall time: 37min 23s


In [11]:
file_name = f"imf_{VER}_for_train_or_eval.parquet"
candidates.to_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task2/" + file_name)

In [12]:
candidates.head()

,session_id,candidate_item,imf_score_to_recent_intereaction_n2,imf_rank_to_recent_intereaction_n2
0,test_leftover_0,B06XGHV89F,0.959849,1
1,test_leftover_0,B07MCFD1MK,0.673939,2
2,test_leftover_0,B07Z3L1NN2,0.666873,3
3,test_leftover_0,B07HRGNG8K,0.603244,4
4,test_leftover_0,B01M3RT7MT,0.596283,5


## MRR@100

In [13]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task2/train_task2.parquet")
candidates = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task2/" + file_name)

In [14]:
# candidateの結合とlabelの付与
df = train.join(candidates, on="session_id", how="left")
df = df.sort(["session_id", "imf_score_to_recent_intereaction_n2"], descending=[False, True])
df = df.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
label_lists = df.groupby("session_id", maintain_order=True).all()["label"].to_list()

In [15]:
# MRRの計算
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.22281


# For test inference

In [16]:
train = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet")
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet").filter(pl.col("locale").is_in(LOCALES))
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test2_1, test2_2, test3_1, test3_2])

In [17]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [18]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])
session_df = session_df.to_pandas()

In [19]:
%%time
candidates = train_and_generate_candidates(session_df)

1978362it [01:49, 18074.46it/s]
911508it [00:50, 17874.07it/s]


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 455754/455754 [31:19<00:00, 242.43it/s]


CPU times: user 2h 45min 43s, sys: 1h 41min 30s, total: 4h 27min 13s
Wall time: 38min 18s


In [20]:
file_name = f"imf_{VER}_for_inference.parquet"
candidates.to_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task2/" + file_name)

In [21]:
candidates.head()

,session_id,candidate_item,imf_score_to_recent_intereaction_n2,imf_rank_to_recent_intereaction_n2
0,test_phase1_0,B08GYKNCCP,1.000891,1
1,test_phase1_0,B08GY8NHF2,0.682636,2
2,test_phase1_0,B08ZJMPY2T,0.610372,3
3,test_phase1_0,B08GYG5SVQ,0.584222,4
4,test_phase1_0,B08GYBBBBH,0.533170,5
